# Explore here

In [ ]:
# imports:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# split para modelado
from sklearn.model_selection import train_test_split
# Scaled | Escalado
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Encoding | Codificación
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
# To save models
import json
import pickle
# Feature Selection
from sklearn.feature_selection import f_classif, SelectKBest

In [ ]:
df = pd.read_csv("../data/raw/diabetes-data.csv")
df.head()

* **Planteamiento:** Este conjunto de datos proviene originalmente del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. 
* **Objetivo:** Predecir en base a medidas diagnósticas si un paciente tiene o no diabetes.

In [ ]:
df.info() # explorar datos

**Observaciones:**

> DIMENSIONES DEL DATASET Y TIPOS DE DATO:
> * Contiene 768 filas y 9 columnas  
> * Los datos que aporta el datset son diez columnas de tipo numérico (enteros y decimales) y séis columnas de tipo string (cadenas de texto)  
> * ``dtypes: float64(3), int64(7), object(6)`

In [ ]:
df.duplicated().sum() # * **Valores nulos y duplicados:**
df.isna().sum()

Importante tener en cuenta que tratamos datos médicos y no podemos inventar o elminar información que no este justificada ya que queremos obtener el minimo error en el resultado ya que se trata de la salud de una persona

ELEGIR NUMERO DE GLUCOSA PARA HACER DIVISION. ¿ A PARTIR DE QUE VALORES DE 'glusoca','insulin'... EL PACIENTE TIENE DIABETES?

In [ ]:
# Fixing columns with missing values
data_zero = ["SkinThickness", "Insulin", "BloodPressure"]

# Replazing zeros with mean of each column
for column in data_zero:
    median_value = df[df[column] != 0][column].median()
    df[column] = df[column].replace(0, median_value)


df[df['Insulin']==0].sum()

In [ ]:
df = df.drop(df[(df['BMI'] == 0) & (df['Glucose'] == 0)].index) # eliminar filas a 0

VARIABLE MEJOR PREDICTORA ES GLUCOSA, YA QUE DETERMINA SI TIENES DIABETES O NO.

**Conclusiones:** DUDA DOUBLE CHECK

* Se reemplazan los valores a 0 con la media ya que trabaja mejor cuando hay más cantidad de outliers para determinar el valor central de toda la columna.  
"SkinThickness", "Insulin", "BloodPressure":  

* Filtrado de filas: Hay columnas que todavía contienen valores a 0. Son Glucosa y BMI.  
Glucosa, normalmente no es posible que el resultado sea cerro y se considera un error asi que alimino esas filas  
BMI, no es posible que una persona pese 0kg así que elmino también esas filas

In [ ]:
df.shape # Después del filtrado

**Varibales independientes**

COLUMNAS Y CARACTERÍSTICAS DE LAS VARIABLES: 

1. NUMÉRICAS:

> Continuas: Medicas
- `Glucose`. Concentración de glucosa en plasma a las 2 horas de un test de tolerancia oral a la glucosa (numérico)
- `BloodPressure`. Presión arterial diastólica (medida en mm Hg) (numérico)
- `SkinThickness`. Grosor del pliegue cutáneo del tríceps (medida en mm) (numérico)
- `Insulin`. Insulina sérica de 2 horas (medida en mu U/ml) (numérico)
- `BMI`. Índice de masa corporal (numérico)

> Discretas: Personas
- `Pregnancies`. Número de embarazos del paciente (numérico)
- `DiabetesPedigreeFunction`. Función de pedigrí de diabetes (numérico)
- `Age`. Edad del paciente (numérico)

2. CATEGÓRICAS  
No hay columnas co características categóricas

**Variable target** 
> ``Outcome``, es la variable que nos da el resultado del planteamiento del problema, si un paciente es diabetico o no. No se indica el tipo de diabetes de la persona.

In [ ]:
# Gráficos
# Separo las variables en grupo 
medical = ['DiabetesPedigreeFunction', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
person = ['Pregnancies', 'DiabetesPedigreeFunction', 'Age']

In [ ]:
# La barra lateral represnta el conteo de datos con parámetro 'cbar'
import math 
# Gráficos para la información médica
n = len(medical)
cols_grid = 3                         
rows = math.ceil(n / cols_grid)   # math.ceil() calcula una divisón  y redondea a un numero entero
fig, axis = plt.subplots(rows,cols_grid, figsize=(12, 7))
axis = axis.flatten()
for i, columna in enumerate(medical):
    sns.histplot(data=df, x="Outcome", y=columna, cbar=True,
        ax=axis[i]
    )
    axis[i].set_title(f"{columna} - Target")

plt.tight_layout()
plt.show()


**Observaciones:**
* ``BMI``, niveles altos indican mayor resistencia a la insulina y diabetes de tipo 2 (índice de masa corporal). En el gráfico los valores se acumulan entre el 21 y 40  
Según rangos estándar OMS, a partir del 30 oo más se considera obesidad y este si es un factor que aumenta el reisgo a ser diabetico

 * ``Skinthickness``, según el grosos de la piel podemos detectar el nivel de grasa (obesidad del paciente) que es una condición que puede influir en la diabetes

 * ``BloodPressure``, relacionado con BMI, la resistencia a la insulina (absorción baja de glucosa) aumenta los niveles de presión arterial

 * ``Glucose``, los resultados de un test de tolerancia a la glucosa determinan si un paciente es diabetico o prediabetico. La mayor concentración de data esta entre el 80-130.  
 La mayoria de los diagnosticados sin diabetes según los rangos de glucosa podrían ser prediabeticos ya que lo normal sería menos de 100 de glucosa en sangre

 * ``Insulin``, la insulina sérica es la insulina presente en la sangre.Si los niveles son bajos entonces indica deficit de produccion de glucosa por lo tanto puede tratarse de diabetes avanzada  
 La insulina es la hormonoa que regula la absorción de la glucosa, por eso cuando es baja se presdenta como síntoma de diabetes.


In [ ]:
# Gráficos para la información personal
fig, axis = plt.subplots(1, 3, figsize=(10, 3))   

for i, columna in enumerate(person):          
    sns.histplot(data=df, x="Outcome", y=columna, cbar=True, ax=axis[i])
    axis[i].set_title(f"{columna} - Target")

plt.tight_layout()
plt.show()

In [ ]:
# Gráfica general con particiones

features = df.columns.drop("Outcome")

for col in features:
    plt.figure()
    df.boxplot(column=col, by="Outcome")
    plt.title(f"{col} vs Outcome")
    plt.suptitle("")  # remove automatic title
    plt.xlabel("Outcome")
    plt.ylabel(col)
    plt.show()

In [ ]:
# Gráfica general con particiones

for col in df:
    plt.figure()

    df[df["Outcome"] == 0][col].hist(alpha=0.5, label="0")
    df[df["Outcome"] == 1][col].hist(alpha=0.5, label="1")

    plt.title(f"{col} vs Outcome")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()

In [ ]:
for col in df:
    plt.figure()

    y = df["Outcome"] + np.random.normal(0, 0.02, len(df))

    plt.scatter(df[col], y)
    plt.xlabel(col)
    plt.ylabel("Outcome")
    plt.title(f"{col} vs Outcome")
    plt.show()

In [ ]:
means = df.groupby("Outcome").mean()

for col in df:
    plt.figure()
    means[col].plot(kind="bar")
    plt.title(f"Mean {col} by Outcome")
    plt.ylabel("Mean value")
    plt.show()

**Observaciones:**
* ``Pregnancies``, los emabrazos puedes aumentar el riesgo de desarrollar diabetes tipo 2 en el futuro. La mayoria de las personas sí han estado embarazadas
 
* ``DiabetesPedigreeFunction``, riesgo genético de tener diabetes. Hay más concentración de datos en las personas que están diagnosticadas sin la enfermedad tienen indices entre el 0.20 y 0.40

In [ ]:
# Correlaciones

corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

fig, axis = plt.subplots(figsize=(8, 6))
sns.heatmap(corr, mask=mask, annot=True, linewidths=0.5, fmt=".2f")
plt.tight_layout()
plt.show()

**Conclusiones:**
* La columna más relacionada con el target es la Glucosa seguida del peso corporal de la persona
* Ninguna de las columnas de correlaciona altamente así que no elimino ninguna columna

# Saving processed data --> GUARDADO DE LOS DATOS PROCESADOR
df.to_csv("../data/processed/diabetes_eda.csv", index=False)

ML: ARBOLES DE DECISIÓN

In [ ]:
# Split data X & y
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

In [ ]:
# Split data TRaing& Test
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train, y_train, X_test

In [ ]:
#imports
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
mi_prediccion = (X_test['Glucose'] > 120).astype(int)

In [ ]:
plt.figure(figsize=(12,12))
tree.plot_tree(clf, 
               feature_names=["viento",
                              "clima_nublado","clima_soleado",
                              "temperatura_fresco", "temperatura_templado",
                              "olas_pocas"],  
               class_names=["no","si"],
               filled=True);

In [ ]:
predicciones_val = dt.predict(X_val)
accuracy = sklearn.metrics.accuracy_score(y_val, predicciones_val)
print("Accuracy Arbol:", accuracy)